In [1]:
![ -d vision_transformer ] || git clone --depth=1 https://github.com/google-research/vision_transformer

In [2]:
!pip install -qr vision_transformer/vit_jax/requirements.txt

In [3]:
# Import files from repository.

# Provide OOM errors
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import sys
if './vision_transformer' not in sys.path:
  sys.path.append('./vision_transformer')


from tqdm.auto import tqdm
import tensorflow as tf
import jax
from torchvision import transforms as T
from torchvision import datasets
import torch as th

from vit_jax import checkpoint
from vit_jax import models
from vit_jax.configs import models as models_config

2022-11-17 19:15:34.498166: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 19:15:34.641474: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-17 19:15:35.247541: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-17 19:15:35.247620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [4]:
def get_run_model_jit(model, params):
  return jax.jit(lambda x: model.apply({"params": params}, x, train=False))

def get_accuracy(model, params, dataset, batch_size):
  run_model_jit = get_run_model_jit(model, params)
  dataloader = th.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

  good = 0
  total = 0
  tqdm_loader = tqdm(dataloader, desc="Evaluating", unit="batches")
  for images, labels in tqdm_loader:
    images = images.permute(0, 2, 3, 1).numpy()
    logits = run_model_jit(images)

    good  += jax.device_get(jax.numpy.equal(logits.argmax(axis=1), labels.numpy()).sum())
    total += labels.shape[0]

    tqdm_loader.set_description_str(f"Accuracy: {good / total:.2%}")

  return good / total

In [5]:
transforms = T.Compose([
  T.ToTensor(),
  T.Resize((384, 384)),
])

dataset = datasets.ImageNet(root="ImageNet", split="val", transform=transforms)

In [6]:
model_config = models_config.MODEL_CONFIGS["ViT-B_16"]
path = "gs://vit_models/imagenet21k+imagenet2012/ViT-B_16.npz"
print(f'{tf.io.gfile.stat(path).length / 1024 / 1024:.1f} MiB - {path}')

model = models.VisionTransformer(num_classes=1000, **model_config)
params = checkpoint.load(path)
accuracy_b_16 = get_accuracy(model, params, dataset, 8)
print(f"ViT-B_16 w/ Imagenet21k accuracy: {accuracy_b_16:.2%}")

2022-11-17 19:15:37.421818: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


331.4 MiB - gs://vit_models/imagenet21k+imagenet2012/ViT-B_16.npz


Evaluating:   0%|          | 0/6250 [00:00<?, ?batches/s]

ViT-B_16 w/ Imagenet21k accuracy: 81.56%


In [7]:
model_config = models_config.MODEL_CONFIGS["ViT-L_16"]
path = "gs://vit_models/imagenet21k+imagenet2012/ViT-L_16.npz"
print(f'{tf.io.gfile.stat(path).length / 1024 / 1024:.1f} MiB - {path}')

model = models.VisionTransformer(num_classes=1000, **model_config)
params = checkpoint.load(path)
accuracy_b_16 = get_accuracy(model, params, dataset, 8)
print(f"ViT-L_16 w/ Imagenet21k accuracy: {accuracy_b_16:.2%}")

1162.5 MiB - gs://vit_models/imagenet21k+imagenet2012/ViT-L_16.npz


Evaluating:   0%|          | 0/6250 [00:00<?, ?batches/s]

KeyboardInterrupt: 